In [1]:
import tensorflow as tf
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Using ReLU and reconstruction with 3 hidden layers
### Tensorflow GradientDescentOptimizer

In [3]:
with tf.Graph().as_default() as g: 

  learning_rate = tf.placeholder(tf.float32, shape=[])

  with tf.name_scope('in') as scope:
    x = tf.placeholder(tf.float32, shape=[None, 784])
    tf.histogram_summary(scope + '/image', x)
    tf.image_summary('input', tf.reshape(x, [-1,28,28,1]), 3)

  with tf.name_scope('hidden_1') as scope:
    W = tf.Variable(tf.truncated_normal([784,100], stddev=0.1))
    b = tf.Variable(tf.truncated_normal([100], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W)
    tf.histogram_summary(scope + '/biases', b)
    a = tf.nn.relu(tf.matmul(x, W) + b)
    tf.histogram_summary(scope + '/activations', a)
  with tf.name_scope('hidden_2') as scope:
    W_2 = tf.Variable(tf.truncated_normal([100,100], stddev=0.1))
    b_2 = tf.Variable(tf.truncated_normal([100], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W_2)
    tf.histogram_summary(scope + '/biases', b_2)
    a_2 = tf.nn.relu(tf.matmul(a, W_2) + b_2)
    tf.histogram_summary(scope + '/activations', a_2)
  with tf.name_scope('hidden_3') as scope:
    W_3 = tf.Variable(tf.truncated_normal([100,100], stddev=0.1))
    b_3 = tf.Variable(tf.truncated_normal([100], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W_3)
    tf.histogram_summary(scope + '/biases', b_3)
    a_3 = tf.nn.relu(tf.matmul(a_2, W_3) + b_3)
    tf.histogram_summary(scope + '/activations', a_3)
  with tf.name_scope('out') as scope:
    y_ = tf.placeholder(tf.float32, shape=[None, 10])
    W_4 = tf.Variable(tf.truncated_normal([100,10], stddev=0.1))
    b_4 = tf.Variable(tf.truncated_normal([10], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W_4)
    tf.histogram_summary(scope + '/biases', b_4)
    y = tf.nn.sigmoid(tf.matmul(a_3, W_4) + b_4)
    tf.histogram_summary(scope + '/activations', y)

  #     with tf.name_scope('hidden_2') as scope:
  #       a_2_r = tf.nn.relu(tf.matmul(a_3 - b_3, tf.transpose(W_3)))
  #       tf.histogram_summary(scope + '/recons_activations', a_2_r)
  #     with tf.name_scope('hidden_1') as scope:
  #       a_r = tf.nn.relu(tf.matmul(a_2_r - b_2, tf.transpose(W_2)))
  #       tf.histogram_summary(scope + '/recons_activations', a_r)
  #     with tf.name_scope('in') as scope:
  #       x_r = tf.matmul(a_r - b, tf.transpose(W))
  #       tf.histogram_summary(scope + '/reconstruction', x_r)
  #       tf.image_summary('reconstruction', tf.reshape(x_r, [-1,28,28,1]), 3)

  global_step = tf.Variable(0, name='global_step', trainable=False)

  # losses
  cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y) + (1 - y_) * tf.log(1-y), 
                                                reduction_indices=[1]),
                                 name='xentropy_mean')
  tf.scalar_summary(cross_entropy.op.name, cross_entropy)

  # training
  # train_step = tf.train.GradientDescentOptimizer(1e-6).minimize(cross_entropy)
  train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy, 
                                                            global_step=global_step)
  
  #     back_train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(reconstruction_error)

  # evaluation
  correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  ########## INIT AND SUMMARY HAVE TO APPEAR HERE AT THE END TO BE PROPERLY DEFINED
  # misc
  init = tf.initialize_all_variables()
  summary_op = tf.merge_all_summaries()

  with tf.Session() as sess:

    sess.run(init)
    summary_writer = tf.train.SummaryWriter('./mnist_own_descent/tf_grad', sess.graph)

    num_epochs = 10
    steps_per_epoch = 10000
    for epoch in range(num_epochs):
      l_r = 10**np.linspace(-4,-7,num_epochs)[epoch]
      print("epoch %d, learning rate %g"%(epoch,l_r))

      for step in range(steps_per_epoch):
        batch = mnist.train.next_batch(50)
        feed_dict = {x: batch[0], y_: batch[1], learning_rate: l_r}

  #           if step%1000 == 0:
  # #             reconstruction = x_r.eval(feed_dict=feed_dict)
  # #             print("Reconstruction of a %d"%(np.argmax(batch[1][0])))
  # #             plt.imshow(np.reshape(reconstruction[0],(28,28)),
  # #                        cmap=plt.get_cmap('gray'),
  # #                        interpolation='none',
  # #                        aspect="auto")
  # #             plt.show()

        if step%100 == 0:
          train_accuracy = accuracy.eval(feed_dict=feed_dict)
          print("step %d, train accuracy %g"%(step + epoch*steps_per_epoch, train_accuracy))
          summary_str = sess.run(summary_op, feed_dict=feed_dict)
          summary_writer.add_summary(summary_str, step + epoch*steps_per_epoch)
          summary_writer.flush()

        train_step.run(feed_dict=feed_dict)
  #           back_train_step.run(feed_dict=feed_dict)

    print("test accuracy %g"%accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels}))

epoch 0, learning rate 0.0001
step 0, train accuracy 0.06
step 100, train accuracy 0.06
step 200, train accuracy 0.1
step 300, train accuracy 0.1
step 400, train accuracy 0.12
step 500, train accuracy 0.14
step 600, train accuracy 0.1
step 700, train accuracy 0.04
step 800, train accuracy 0.06
step 900, train accuracy 0.1
step 1000, train accuracy 0.08
step 1100, train accuracy 0.1
step 1200, train accuracy 0.04
step 1300, train accuracy 0.06
step 1400, train accuracy 0.12
step 1500, train accuracy 0.08
step 1600, train accuracy 0.14
step 1700, train accuracy 0.14
step 1800, train accuracy 0.08
step 1900, train accuracy 0.2
step 2000, train accuracy 0.08
step 2100, train accuracy 0.12
step 2200, train accuracy 0.08
step 2300, train accuracy 0.16
step 2400, train accuracy 0.2
step 2500, train accuracy 0.22
step 2600, train accuracy 0.16
step 2700, train accuracy 0.14
step 2800, train accuracy 0.24
step 2900, train accuracy 0.08
step 3000, train accuracy 0.1
step 3100, train accuracy 0.1

In [5]:
with tf.Graph().as_default() as g: 

  learning_rate = tf.placeholder(tf.float32, shape=[])

  with tf.name_scope('in') as scope:
    x = tf.placeholder(tf.float32, shape=[None, 784])
    tf.histogram_summary(scope + '/image', x)
    tf.image_summary('input', tf.reshape(x, [-1,28,28,1]), 3)

  with tf.name_scope('hidden_1') as scope:
    W = tf.Variable(tf.truncated_normal([784,100], stddev=0.1))
    b = tf.Variable(tf.truncated_normal([100], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W)
    tf.histogram_summary(scope + '/biases', b)
    a = tf.nn.relu(tf.matmul(x, W) + b)
    tf.histogram_summary(scope + '/activations', a)
  with tf.name_scope('hidden_2') as scope:
    W_2 = tf.Variable(tf.truncated_normal([100,100], stddev=0.1))
    b_2 = tf.Variable(tf.truncated_normal([100], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W_2)
    tf.histogram_summary(scope + '/biases', b_2)
    a_2 = tf.nn.relu(tf.matmul(a, W_2) + b_2)
    tf.histogram_summary(scope + '/activations', a_2)
  with tf.name_scope('hidden_3') as scope:
    W_3 = tf.Variable(tf.truncated_normal([100,100], stddev=0.1))
    b_3 = tf.Variable(tf.truncated_normal([100], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W_3)
    tf.histogram_summary(scope + '/biases', b_3)
    a_3 = tf.nn.relu(tf.matmul(a_2, W_3) + b_3)
    tf.histogram_summary(scope + '/activations', a_3)
  with tf.name_scope('out') as scope:
    y_ = tf.placeholder(tf.float32, shape=[None, 10])
    W_4 = tf.Variable(tf.truncated_normal([100,10], stddev=0.1))
    b_4 = tf.Variable(tf.truncated_normal([10], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W_4)
    tf.histogram_summary(scope + '/biases', b_4)
    y = tf.nn.sigmoid(tf.matmul(a_3, W_4) + b_4)
    tf.histogram_summary(scope + '/activations', y)

  #     with tf.name_scope('hidden_2') as scope:
  #       a_2_r = tf.nn.relu(tf.matmul(a_3 - b_3, tf.transpose(W_3)))
  #       tf.histogram_summary(scope + '/recons_activations', a_2_r)
  #     with tf.name_scope('hidden_1') as scope:
  #       a_r = tf.nn.relu(tf.matmul(a_2_r - b_2, tf.transpose(W_2)))
  #       tf.histogram_summary(scope + '/recons_activations', a_r)
  #     with tf.name_scope('in') as scope:
  #       x_r = tf.matmul(a_r - b, tf.transpose(W))
  #       tf.histogram_summary(scope + '/reconstruction', x_r)
  #       tf.image_summary('reconstruction', tf.reshape(x_r, [-1,28,28,1]), 3)

  global_step = tf.Variable(0, name='global_step', trainable=False)

  # losses
  cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y) + (1 - y_) * tf.log(1-y), 
                                                reduction_indices=[1]),
                                 name='xentropy_mean')
  tf.scalar_summary(cross_entropy.op.name, cross_entropy)

  # training
  # train_step = tf.train.GradientDescentOptimizer(1e-6).minimize(cross_entropy)
  train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy, 
                                                            global_step=global_step)
  
  #     back_train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(reconstruction_error)

  # evaluation
  correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  ########## INIT AND SUMMARY HAVE TO APPEAR HERE AT THE END TO BE PROPERLY DEFINED
  # misc
  init = tf.initialize_all_variables()
  summary_op = tf.merge_all_summaries()

  with tf.Session() as sess:

    sess.run(init)
    summary_writer = tf.train.SummaryWriter('./mnist_own_descent/tf_adam', sess.graph)

    num_epochs = 10
    steps_per_epoch = 10000
    for epoch in range(num_epochs):
      l_r = 10**np.linspace(-5,-8,num_epochs)[epoch]
      print("epoch %d, learning rate %g"%(epoch,l_r))

      for step in range(steps_per_epoch):
        batch = mnist.train.next_batch(50)
        feed_dict = {x: batch[0], y_: batch[1], learning_rate: l_r}

  #           if step%1000 == 0:
  # #             reconstruction = x_r.eval(feed_dict=feed_dict)
  # #             print("Reconstruction of a %d"%(np.argmax(batch[1][0])))
  # #             plt.imshow(np.reshape(reconstruction[0],(28,28)),
  # #                        cmap=plt.get_cmap('gray'),
  # #                        interpolation='none',
  # #                        aspect="auto")
  # #             plt.show()

        if step%100 == 0:
          train_accuracy = accuracy.eval(feed_dict=feed_dict)
          print("step %d, train accuracy %g"%(step + epoch*steps_per_epoch, train_accuracy))
          summary_str = sess.run(summary_op, feed_dict=feed_dict)
          summary_writer.add_summary(summary_str, step + epoch*steps_per_epoch)
          summary_writer.flush()

        train_step.run(feed_dict=feed_dict)
  #           back_train_step.run(feed_dict=feed_dict)

    print("test accuracy %g"%accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels}))

epoch 0, learning rate 1e-05
step 0, train accuracy 0.02
step 100, train accuracy 0.08
step 200, train accuracy 0.18
step 300, train accuracy 0.16
step 400, train accuracy 0.08
step 500, train accuracy 0.12
step 600, train accuracy 0.1
step 700, train accuracy 0.14
step 800, train accuracy 0.16
step 900, train accuracy 0.18
step 1000, train accuracy 0.2
step 1100, train accuracy 0.24
step 1200, train accuracy 0.28
step 1300, train accuracy 0.24
step 1400, train accuracy 0.36
step 1500, train accuracy 0.46
step 1600, train accuracy 0.46
step 1700, train accuracy 0.52
step 1800, train accuracy 0.52
step 1900, train accuracy 0.58
step 2000, train accuracy 0.62
step 2100, train accuracy 0.66
step 2200, train accuracy 0.62
step 2300, train accuracy 0.58
step 2400, train accuracy 0.52
step 2500, train accuracy 0.6
step 2600, train accuracy 0.54
step 2700, train accuracy 0.68
step 2800, train accuracy 0.64
step 2900, train accuracy 0.74
step 3000, train accuracy 0.72
step 3100, train accuracy

In [6]:
with tf.Graph().as_default() as g: 

  learning_rate = tf.placeholder(tf.float32, shape=[])

  with tf.name_scope('in') as scope:
    x = tf.placeholder(tf.float32, shape=[None, 784])
    tf.histogram_summary(scope + '/image', x)
    tf.image_summary('input', tf.reshape(x, [-1,28,28,1]), 3)

  with tf.name_scope('hidden_1') as scope:
    W = tf.Variable(tf.truncated_normal([784,100], stddev=0.1))
    b = tf.Variable(tf.truncated_normal([100], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W)
    tf.histogram_summary(scope + '/biases', b)
    a = tf.nn.relu(tf.matmul(x, W) + b)
    tf.histogram_summary(scope + '/activations', a)
  with tf.name_scope('hidden_2') as scope:
    W_2 = tf.Variable(tf.truncated_normal([100,100], stddev=0.1))
    b_2 = tf.Variable(tf.truncated_normal([100], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W_2)
    tf.histogram_summary(scope + '/biases', b_2)
    a_2 = tf.nn.relu(tf.matmul(a, W_2) + b_2)
    tf.histogram_summary(scope + '/activations', a_2)
  with tf.name_scope('hidden_3') as scope:
    W_3 = tf.Variable(tf.truncated_normal([100,100], stddev=0.1))
    b_3 = tf.Variable(tf.truncated_normal([100], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W_3)
    tf.histogram_summary(scope + '/biases', b_3)
    a_3 = tf.nn.relu(tf.matmul(a_2, W_3) + b_3)
    tf.histogram_summary(scope + '/activations', a_3)
  with tf.name_scope('out') as scope:
    y_ = tf.placeholder(tf.float32, shape=[None, 10])
    W_4 = tf.Variable(tf.truncated_normal([100,10], stddev=0.1))
    b_4 = tf.Variable(tf.truncated_normal([10], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W_4)
    tf.histogram_summary(scope + '/biases', b_4)
    y = tf.nn.sigmoid(tf.matmul(a_3, W_4) + b_4)
    tf.histogram_summary(scope + '/activations', y)

  #     with tf.name_scope('hidden_2') as scope:
  #       a_2_r = tf.nn.relu(tf.matmul(a_3 - b_3, tf.transpose(W_3)))
  #       tf.histogram_summary(scope + '/recons_activations', a_2_r)
  #     with tf.name_scope('hidden_1') as scope:
  #       a_r = tf.nn.relu(tf.matmul(a_2_r - b_2, tf.transpose(W_2)))
  #       tf.histogram_summary(scope + '/recons_activations', a_r)
  #     with tf.name_scope('in') as scope:
  #       x_r = tf.matmul(a_r - b, tf.transpose(W))
  #       tf.histogram_summary(scope + '/reconstruction', x_r)
  #       tf.image_summary('reconstruction', tf.reshape(x_r, [-1,28,28,1]), 3)

  global_step = tf.Variable(0, name='global_step', trainable=False)

  # losses
  cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y) + (1 - y_) * tf.log(1-y), 
                                                reduction_indices=[1]),
                                 name='xentropy_mean')
  tf.scalar_summary(cross_entropy.op.name, cross_entropy)

  # training
  # train_step = tf.train.GradientDescentOptimizer(1e-6).minimize(cross_entropy)
#   train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy, 
#                                                             global_step=global_step)
  opt = tf.train.GradientDescentOptimizer(learning_rate)
  var_list = [W,b, W_2,b_2, W_3,b_3, W_4,b_4]
  var_grad = tf.gradients(cross_entropy, var_list)
  grads_and_vars = zip(var_grad, var_list)
  train_step = opt.apply_gradients(grads_and_vars, 
                                   global_step=global_step)
  
  #     back_train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(reconstruction_error)

  # evaluation
  correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  ########## INIT AND SUMMARY HAVE TO APPEAR HERE AT THE END TO BE PROPERLY DEFINED
  # misc
  init = tf.initialize_all_variables()
  summary_op = tf.merge_all_summaries()

  with tf.Session() as sess:

    sess.run(init)
    summary_writer = tf.train.SummaryWriter('./mnist_own_descent/own_grad', sess.graph)

    num_epochs = 10
    steps_per_epoch = 10000
    for epoch in range(num_epochs):
      l_r = 10**np.linspace(-4,-7,num_epochs)[epoch]
      print("epoch %d, learning rate %g"%(epoch,l_r))

      for step in range(steps_per_epoch):
        batch = mnist.train.next_batch(50)
        feed_dict = {x: batch[0], y_: batch[1], learning_rate: l_r}

  #           if step%1000 == 0:
  # #             reconstruction = x_r.eval(feed_dict=feed_dict)
  # #             print("Reconstruction of a %d"%(np.argmax(batch[1][0])))
  # #             plt.imshow(np.reshape(reconstruction[0],(28,28)),
  # #                        cmap=plt.get_cmap('gray'),
  # #                        interpolation='none',
  # #                        aspect="auto")
  # #             plt.show()

        if step%100 == 0:
          train_accuracy = accuracy.eval(feed_dict=feed_dict)
          print("step %d, train accuracy %g"%(step + epoch*steps_per_epoch, train_accuracy))
          summary_str = sess.run(summary_op, feed_dict=feed_dict)
          summary_writer.add_summary(summary_str, step + epoch*steps_per_epoch)
          summary_writer.flush()

        train_step.run(feed_dict=feed_dict)
  #           back_train_step.run(feed_dict=feed_dict)

    print("test accuracy %g"%accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels}))

epoch 0, learning rate 0.0001
step 0, train accuracy 0.06
step 100, train accuracy 0.1
step 200, train accuracy 0.1
step 300, train accuracy 0.1
step 400, train accuracy 0.06
step 500, train accuracy 0.06
step 600, train accuracy 0.06
step 700, train accuracy 0.16
step 800, train accuracy 0.04
step 900, train accuracy 0.08
step 1000, train accuracy 0.02
step 1100, train accuracy 0.1
step 1200, train accuracy 0.1
step 1300, train accuracy 0.14
step 1400, train accuracy 0.08
step 1500, train accuracy 0.12
step 1600, train accuracy 0.14
step 1700, train accuracy 0.12
step 1800, train accuracy 0.12
step 1900, train accuracy 0.1
step 2000, train accuracy 0.18
step 2100, train accuracy 0.16
step 2200, train accuracy 0.12
step 2300, train accuracy 0.1
step 2400, train accuracy 0.04
step 2500, train accuracy 0.16
step 2600, train accuracy 0.02
step 2700, train accuracy 0.14
step 2800, train accuracy 0.06
step 2900, train accuracy 0.14
step 3000, train accuracy 0.08
step 3100, train accuracy 0.

In [7]:
with tf.Graph().as_default() as g: 

  learning_rate = tf.placeholder(tf.float32, shape=[])

  with tf.name_scope('in') as scope:
    x = tf.placeholder(tf.float32, shape=[None, 784])
    tf.histogram_summary(scope + '/image', x)
    tf.image_summary('input', tf.reshape(x, [-1,28,28,1]), 3)

  with tf.name_scope('hidden_1') as scope:
    W = tf.Variable(tf.truncated_normal([784,100], stddev=0.1))
    b = tf.Variable(tf.truncated_normal([100], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W)
    tf.histogram_summary(scope + '/biases', b)
    a = tf.nn.relu(tf.matmul(x, W) + b)
    tf.histogram_summary(scope + '/activations', a)
  with tf.name_scope('hidden_2') as scope:
    W_2 = tf.Variable(tf.truncated_normal([100,100], stddev=0.1))
    b_2 = tf.Variable(tf.truncated_normal([100], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W_2)
    tf.histogram_summary(scope + '/biases', b_2)
    a_2 = tf.nn.relu(tf.matmul(a, W_2) + b_2)
    tf.histogram_summary(scope + '/activations', a_2)
  with tf.name_scope('hidden_3') as scope:
    W_3 = tf.Variable(tf.truncated_normal([100,100], stddev=0.1))
    b_3 = tf.Variable(tf.truncated_normal([100], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W_3)
    tf.histogram_summary(scope + '/biases', b_3)
    a_3 = tf.nn.relu(tf.matmul(a_2, W_3) + b_3)
    tf.histogram_summary(scope + '/activations', a_3)
  with tf.name_scope('out') as scope:
    y_ = tf.placeholder(tf.float32, shape=[None, 10])
    W_4 = tf.Variable(tf.truncated_normal([100,10], stddev=0.1))
    b_4 = tf.Variable(tf.truncated_normal([10], stddev=0.1))
    tf.histogram_summary(scope + '/weights', W_4)
    tf.histogram_summary(scope + '/biases', b_4)
    y = tf.nn.sigmoid(tf.matmul(a_3, W_4) + b_4)
    tf.histogram_summary(scope + '/activations', y)

  #     with tf.name_scope('hidden_2') as scope:
  #       a_2_r = tf.nn.relu(tf.matmul(a_3 - b_3, tf.transpose(W_3)))
  #       tf.histogram_summary(scope + '/recons_activations', a_2_r)
  #     with tf.name_scope('hidden_1') as scope:
  #       a_r = tf.nn.relu(tf.matmul(a_2_r - b_2, tf.transpose(W_2)))
  #       tf.histogram_summary(scope + '/recons_activations', a_r)
  #     with tf.name_scope('in') as scope:
  #       x_r = tf.matmul(a_r - b, tf.transpose(W))
  #       tf.histogram_summary(scope + '/reconstruction', x_r)
  #       tf.image_summary('reconstruction', tf.reshape(x_r, [-1,28,28,1]), 3)

  global_step = tf.Variable(0, name='global_step', trainable=False)

  # losses
  cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y) + (1 - y_) * tf.log(1-y), 
                                                reduction_indices=[1]),
                                 name='xentropy_mean')
  tf.scalar_summary(cross_entropy.op.name, cross_entropy)

  # training
  # train_step = tf.train.GradientDescentOptimizer(1e-6).minimize(cross_entropy)
#   train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy, 
#                                                             global_step=global_step)
  opt = tf.train.AdamOptimizer(learning_rate)
  var_list = [W,b, W_2,b_2, W_3,b_3, W_4,b_4]
  var_grad = tf.gradients(cross_entropy, var_list)
  grads_and_vars = zip(var_grad, var_list)
  train_step = opt.apply_gradients(grads_and_vars, 
                                   global_step=global_step)
  
  #     back_train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(reconstruction_error)

  # evaluation
  correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  ########## INIT AND SUMMARY HAVE TO APPEAR HERE AT THE END TO BE PROPERLY DEFINED
  # misc
  init = tf.initialize_all_variables()
  summary_op = tf.merge_all_summaries()

  with tf.Session() as sess:

    sess.run(init)
    summary_writer = tf.train.SummaryWriter('./mnist_own_descent/own_adam', sess.graph)

    num_epochs = 10
    steps_per_epoch = 10000
    for epoch in range(num_epochs):
      l_r = 10**np.linspace(-5,-8,num_epochs)[epoch]
      print("epoch %d, learning rate %g"%(epoch,l_r))

      for step in range(steps_per_epoch):
        batch = mnist.train.next_batch(50)
        feed_dict = {x: batch[0], y_: batch[1], learning_rate: l_r}

  #           if step%1000 == 0:
  # #             reconstruction = x_r.eval(feed_dict=feed_dict)
  # #             print("Reconstruction of a %d"%(np.argmax(batch[1][0])))
  # #             plt.imshow(np.reshape(reconstruction[0],(28,28)),
  # #                        cmap=plt.get_cmap('gray'),
  # #                        interpolation='none',
  # #                        aspect="auto")
  # #             plt.show()

        if step%100 == 0:
          train_accuracy = accuracy.eval(feed_dict=feed_dict)
          print("step %d, train accuracy %g"%(step + epoch*steps_per_epoch, train_accuracy))
          summary_str = sess.run(summary_op, feed_dict=feed_dict)
          summary_writer.add_summary(summary_str, step + epoch*steps_per_epoch)
          summary_writer.flush()

        train_step.run(feed_dict=feed_dict)
  #           back_train_step.run(feed_dict=feed_dict)

    print("test accuracy %g"%accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels}))

epoch 0, learning rate 1e-05
step 0, train accuracy 0.16
step 100, train accuracy 0.2
step 200, train accuracy 0.08
step 300, train accuracy 0.22
step 400, train accuracy 0.16
step 500, train accuracy 0.02
step 600, train accuracy 0.1
step 700, train accuracy 0.18
step 800, train accuracy 0.18
step 900, train accuracy 0.18
step 1000, train accuracy 0.34
step 1100, train accuracy 0.2
step 1200, train accuracy 0.3
step 1300, train accuracy 0.4
step 1400, train accuracy 0.38
step 1500, train accuracy 0.3
step 1600, train accuracy 0.34
step 1700, train accuracy 0.44
step 1800, train accuracy 0.48
step 1900, train accuracy 0.5
step 2000, train accuracy 0.54
step 2100, train accuracy 0.56
step 2200, train accuracy 0.58
step 2300, train accuracy 0.7
step 2400, train accuracy 0.58
step 2500, train accuracy 0.54
step 2600, train accuracy 0.54
step 2700, train accuracy 0.66
step 2800, train accuracy 0.7
step 2900, train accuracy 0.9
step 3000, train accuracy 0.76
step 3100, train accuracy 0.8
st